In [48]:
# Method 1
import numpy as np
from collections import defaultdict

def read_sparse_matrix_method1(filename):
    with open(filename, "r") as f:
        n = int(f.readline().strip())  # Read dimension
        d = np.zeros(n)  # Store diagonal elements
        sparse_rows = defaultdict(list)  # Store non-diagonal elements

        # Collect all elements from the file
        data = {}
        for line in f:
            parts = line.strip().split(',')
            if len(parts) == 3:
                value, row, col = map(float, parts)
                row, col = int(row), int(col)

                # Combine elements with the same indices
                if (row, col) in data:
                    data[(row, col)] += value
                else:
                    data[(row, col)] = value

        # Distribute elements to d and sparse_rows
        for (row, col), value in data.items():
            if row == col:
                d[row] = value  # Diagonal element
            else:
                sparse_rows[row].append((col, value))  # Off-diagonal element

    return n, d, sparse_rows

# Example usage
n, d, sparse_rows = read_sparse_matrix_method1("a_4.txt")

# Verify the size
print(f"Matrix size: {n}")

# Check diagonal values
print("First 10 diagonal values:", d[:10])

# Check non-diagonal elements in a few rows
for i in range(5):
    print(f"Row {i}: {sparse_rows[i]}")

def read_vector(filename):
    with open(filename, "r") as f:
        n = int(f.readline().strip())  # Read vector dimension
        b = np.array([float(f.readline().strip()) for _ in range(n)])  # Read values
    return b

# Example usage
b = read_vector("b_4.txt")

# Verify the size
print(f"Vector size: {len(b)} (should be {n})")

# Print first 10 elements to check correctness
print("First 10 elements of b:", b[:10])

def gauss_seidel_setup(n):
    x = np.zeros(n)  # Initial guess: all zeros
    epsilon = 1e-10  # Precision threshold
    max_iterations = 10000  # Safety limit
    return x, epsilon, max_iterations

# Example usage
x, epsilon, max_iterations = gauss_seidel_setup(n)

# Check initial values
print(f"Initial x vector size: {len(x)} (should be {n})")
print(f"Initial x first 10 values: {x[:10]} (should be all zeros)")
print(f"Epsilon (precision threshold): {epsilon}")
print(f"Max iterations: {max_iterations}")

def gauss_seidel(n, d, sparse_rows, b, x, epsilon, max_iterations):
    for iteration in range(max_iterations):
        x_old = x.copy()  # Store previous values to check for convergence

        for i in range(n):
            sum_ax = sum(value * x[col] for col, value in sparse_rows[i])  # Compute sum
            x[i] = (b[i] - sum_ax) / d[i]  # Update x_i

        # Convergence check: max absolute difference between new and old x
        diff = np.max(np.abs(x - x_old))
        if diff < epsilon:
            print(f"Converged in {iteration+1} iterations.")
            return x  # Return final solution

    print("Did not converge within max iterations.")
    return x  # Return best attempt

# Example usage
x_solution = gauss_seidel(n, d, sparse_rows, b, x, epsilon, max_iterations)

# Check first 10 values of x after iterations
print("First 10 values of x:", x_solution[:10])

def verify_solution(n, d, sparse_rows, x_solution, b):
    Ax = np.zeros(n)

    # Compute Ax using the sparse matrix
    for i in range(n):
        Ax[i] = d[i] * x_solution[i] + sum(value * x_solution[col] for col, value in sparse_rows[i])

    # Compute infinity norm ||Ax - b||∞
    error = np.max(np.abs(Ax - b))

    print("Verification ||Ax - b||∞:", error)
    return error

# Run verification
error = verify_solution(n, d, sparse_rows, x_solution, b)


Matrix size: 80000
First 10 diagonal values: [40.5  21.   20.5  81.   49.5  23.25 34.   81.75 51.25 21.75]
Row 0: [(28391, 8.0), (26569, 0.75), (5672, 3.5), (28598, 6.75), (6725, 0.75), (8398, 10.75)]
Row 1: [(29162, 11.0)]
Row 2: [(29388, 8.25), (29449, 2.25)]
Row 3: [(13686, 6.0), (9848, 12.0), (12380, 10.5), (8532, 2.5), (30265, 4.0), (22621, 5.5), (20216, 11.5), (26869, 12.25), (2598, 6.75)]
Row 4: [(7985, 11.25), (21928, 6.25), (8752, 7.5), (27817, 8.25), (15019, 6.25)]
Vector size: 80000 (should be 80000)
First 10 elements of b: [ 5124663.5   2239165.    2171216.75 10983351.75  6503832.75  2611808.5
  4506054.   11361932.5   6767226.5   2568078.75]
Initial x vector size: 80000 (should be 80000)
Initial x first 10 values: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] (should be all zeros)
Epsilon (precision threshold): 1e-10
Max iterations: 10000
Converged in 24 iterations.
First 10 values of x: [79999. 79998. 79997. 79996. 79995. 79994. 79993. 79992. 79991. 79990.]
Verification ||Ax - b||∞: 9.